# 과정 : data에서 6개 컬럼 추출 -> StandardScaler -> KMeans(cluster 번호 매기기) -> 자주 등장하는 행정구 추출 -> 거리 * 인구가 가장 적은 주민센터 확인 및 검토

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#데이터 스케일링, 모델링, 시각화 도구 라이브러리
from sklearn.preprocessing import StandardScaler

In [71]:
data = pd.read_csv('/content/drive/MyDrive/시소프로젝트/1-1 excellent 컷오프 완료.csv', index_col=0)
data.head(1)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,일반,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9
1,종로구,사직동,사직동 주민센터,1.0,2.0,0.0,0.0,50.0,0.0,1087370.0,...,520,1843,6.016724,32.013022,37.576191,126.968829,1.952806e+06,953286.588072,9168.42,62.0


In [72]:
dfnew = data.copy()
dfnew['주변특성'] = None
dfnew['주변특성'] = dfnew['공원수'] + dfnew['도서관수'] + dfnew['병원수'] + dfnew['배움터수']
dfnew.head(1)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
1,종로구,사직동,사직동 주민센터,1.0,2.0,0.0,0.0,50.0,0.0,1087370.0,...,1843,6.016724,32.013022,37.576191,126.968829,1.952806e+06,953286.588072,9168.42,62.0,3.0


In [73]:
df2 = dfnew[['주변특성','대중교통','동별강좌정원','디지털정보지수','독거노인포함','전연령별독거노인비율']]
df2

,주변특성,대중교통,동별강좌정원,디지털정보지수,독거노인포함,전연령별독거노인비율
1,3.0,1.0,50.0,9168.42,1087370.0,6.016724
2,4.0,5.0,360.0,6455.13,2558435.0,15.210701
3,1.0,3.0,0.0,2243.48,161760.0,8.250258
6,3.0,0.0,0.0,7323.87,718820.0,6.263099
7,0.0,3.0,37.0,17054.63,2135562.0,3.982974
...,...,...,...,...,...,...
442,0.0,5.0,170.0,20953.38,2670108.0,3.764684
443,2.0,3.0,15.0,10457.07,2413815.0,6.623742
444,0.0,2.0,443.0,39840.23,5942970.0,3.672402
445,1.0,3.0,489.0,28298.14,943800.0,1.670425


## Standard Scaler

In [74]:
column = df2.columns
column

Index(['주변특성', '대중교통', '동별강좌정원', '디지털정보지수', '독거노인포함', '전연령별독거노인비율'], dtype='object')

In [75]:
scaler = StandardScaler()
dfstan2 = scaler.fit_transform(df2)
dfstan2 = pd.DataFrame(dfstan2, columns = column)
dfstan2.head()

,주변특성,대중교통,동별강좌정원,디지털정보지수,독거노인포함,전연령별독거노인비율
0,1.708000,-0.447988,-0.746355,-1.618131,-1.136415,0.925393
1,2.599597,2.149952,0.196239,-1.905938,-0.659720,5.866992
2,-0.075195,0.850982,-0.898386,-2.352679,-1.436356,2.125877
3,1.708000,-1.097473,-0.898386,-1.813788,-1.255842,1.057815
4,-0.966792,0.850982,-0.785883,-0.781620,-0.796751,-0.167712


## K-means Clustering

In [76]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(dfstan2)
dfstan2['cluster'] = kmeans.labels_
# kmeans.labels_는 각 데이터가 어떤 클러스터에 속하는지 결과를 표현한다.
dfstan2.head()

,주변특성,대중교통,동별강좌정원,디지털정보지수,독거노인포함,전연령별독거노인비율,cluster
0,1.708000,-0.447988,-0.746355,-1.618131,-1.136415,0.925393,2
1,2.599597,2.149952,0.196239,-1.905938,-0.659720,5.866992,2
2,-0.075195,0.850982,-0.898386,-2.352679,-1.436356,2.125877,2
3,1.708000,-1.097473,-0.898386,-1.813788,-1.255842,1.057815,2
4,-0.966792,0.850982,-0.785883,-0.781620,-0.796751,-0.167712,2


In [77]:
dfstan2gr = dfstan2.groupby('cluster')
print(dfstan2.cluster.value_counts())
# 평균으로 상중하 기준 나누기
dfstan2grmean = dfstan2gr.mean()
dfstan2grmean

1    133
2    124
0     75
Name: cluster, dtype: int64


,주변특성,대중교통,동별강좌정원,디지털정보지수,독거노인포함,전연령별독거노인비율
cluster,,,,,,
0,-0.146523,-0.292112,-0.270882,0.998105,1.451972,0.085015
1,-0.396974,-0.355205,0.506726,0.169905,-0.363622,-0.487629
2,0.514410,0.557666,-0.379665,-0.785929,-0.488195,0.471601


In [78]:
# 경고 무시
import warnings
warnings.filterwarnings(action = 'ignore')


In [79]:
# 상중하 코드
for i in range(6):
    max = dfstan2grmean.iloc[:,i].max()
    min = dfstan2grmean.iloc[:,i].min()
    median = dfstan2grmean.iloc[:,i].median()
    for j in range(3):
        if dfstan2grmean.iloc[:,i][j] == min:
            dfstan2grmean.iloc[:,i][j] = '하'
        elif dfstan2grmean.iloc[:,i][j] == max:
            dfstan2grmean.iloc[:,i][j] = '상'
        elif dfstan2grmean.iloc[:,i][j] == median:
            dfstan2grmean.iloc[:,i][j] = '중'

In [80]:
dfstan2grmean

,주변특성,대중교통,동별강좌정원,디지털정보지수,독거노인포함,전연령별독거노인비율
cluster,,,,,,
0,중,중,중,상,상,중
1,하,하,상,중,중,하
2,상,상,하,하,하,상


* 상 = 3점, 중 = 2점, 하 = 1점
* 0번 군집 : 14점
* 1번 군집 : 10점
* 2번 군집 : 12점
* 81명이 있는 군집 채택.
### 0번 군집 채택!!

* 군집에서 가장 많이 차지하는 구 3개 선택, 구에서 P-median 알고리즘으로 거리 * 인구 최소값을 찾고, 현실적으로 그 위치에 방송용 시설을 설치해도 되는지 검증.
* 가정 : 동별강좌정원이 많다는 것은 그만큼 강좌를 수용할 수 있는 공간이 있다고 전제.

## 가정 : 동별강좌정원이 평균이상인 곳으로 입지선정.

In [81]:
# df standard 한 것중 cluster 0번만 index추출
dfnum = dfstan2[dfstan2['cluster']==0]
dfnumindex = dfnum.index
dfnumindex

Int64Index([ 53,  54,  55,  59,  69,  73,  77,  78,  89,  94,  99, 107, 112,
            120, 121, 123, 124, 125, 130, 132, 134, 136, 139, 140, 141, 144,
            145, 147, 148, 152, 158, 161, 163, 171, 176, 180, 181, 183, 185,
            190, 195, 196, 200, 206, 210, 212, 216, 221, 233, 235, 237, 238,
            243, 247, 249, 251, 253, 254, 264, 277, 296, 301, 302, 304, 307,
            308, 309, 315, 319, 320, 321, 322, 323, 326, 329],
           dtype='int64')

In [82]:
# dfnew에서 cluster 0번만 추출.
dfclu2 = dfnew.iloc[dfnumindex]
dfclu2.head(1)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
80,동대문구,용신동,용신동 주민센터,1.0,1.0,0.0,0.0,0.0,0.0,10499223.0,...,5703,5.646893,32.281255,37.575813,127.037289,1.952732e+06,959331.465342,33111.97,86.0,2.0


In [83]:
dfclu2['자치구'].value_counts()

은평구     9
노원구     8
강동구     6
관악구     6
동작구     5
동대문구    5
강서구     5
도봉구     4
중랑구     4
서대문구    3
마포구     3
양천구     3
구로구     3
금천구     3
강북구     2
성북구     2
영등포구    1
서초구     1
강남구     1
송파구     1
Name: 자치구, dtype: int64

## 은평구 시작.

In [84]:
# 은평구
dfeunpyeong = dfclu2[dfclu2['자치구']=='은평구']
dfeunpyeong.head(1)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
177,은평구,녹번동,녹번동 주민센터,0.0,0.0,0.0,0.0,0.0,0.0,12485000.0,...,6875,4.802327,26.414545,37.597487,126.909943,1.955199e+06,948101.674778,36045.69,80.0,0.0


In [85]:
# 인덱스 조정 작업.(117 -> 0)
dfeunpyeong_data = dfeunpyeong.reset_index()
eunpyeong = dfeunpyeong_data.iloc[:,1:]
eunpyeong.head(3)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
0,은평구,녹번동,녹번동 주민센터,0.0,0.0,0.0,0.0,0.0,0.0,12485000.0,...,6875,4.802327,26.414545,37.597487,126.909943,1.955199e+06,948101.674778,36045.69,80.0,0.0
1,은평구,불광1동,불광1동 주민센터,0.0,0.0,0.0,1.0,60.0,0.0,14589827.0,...,7691,4.739893,24.665193,37.610389,126.932113,1.956619e+06,950067.332642,36945.36,88.0,1.0
2,은평구,불광2동,불광2동 주민센터,1.0,0.0,0.0,0.0,70.0,0.0,10161333.0,...,5877,6.016006,29.419772,37.626385,126.927230,1.958396e+06,949647.222171,25945.56,78.0,1.0


In [66]:
eunpyeong.columns

Index(['자치구', '동', '센터명', '공원수', '도서관수', '병원수', '배움터수', '동별강좌정원', '사고건수',
       '독거노인포함', '대중교통', '합계', '일반', '65세이상고령자', '전연령별독거노인비율', '노인인구별독거노인비율',
       'X', 'Y', '위도UTMK', '경도UTMK', '디지털정보지수', '총합 9', '주변특성'],
      dtype='object')

In [86]:
eunpyeong.drop(columns=['공원수','도서관수','병원수','배움터수','일반','X','Y',
                        '총합 9'],inplace=True)

$$
거리합공식 = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}
$$

In [87]:
# 은평구중 각 주민센터(입지선정)와 다른 주민센터와의 거리합
distance = []
distance2 = []
for i in range(len(eunpyeong)):
    for j in range(len(eunpyeong)):
        distance.append(np.sqrt((eunpyeong['위도UTMK'][j] - eunpyeong['위도UTMK'][i])**2 + (eunpyeong['경도UTMK'][j] - eunpyeong['경도UTMK'][i])**2))
    distance2.append(sum(distance))
    distance= []
distance2

[19222.215883903686,
 14894.813103946291,
 17482.143149020947,
 15259.954385819292,
 12666.460144436824,
 21415.973957150924,
 14330.467423818649,
 23395.954420591217,
 15977.276734357478]

In [88]:
# 은평구의 인구 * 거리 값.
# ex) 1행은 녹번동 주민센터에서 각 주민센터마다의 거리 * 녹번동 주민센터의 독거노인포함(가중치열)
# p median 공식 = 수요 * 거리곱의 최소화. 
eunpyeongpopdist = eunpyeong['독거노인포함']*distance2
print(eunpyeongpopdist.sort_values())
print(eunpyeongpopdist.argmin())
eunpyeong.iloc[4:5,:]

4    8.405877e+10
3    1.068376e+11
2    1.776419e+11
5    1.958448e+11
8    1.994763e+11
7    2.020972e+11
1    2.173127e+11
6    2.254678e+11
0    2.399894e+11
Name: 독거노인포함, dtype: float64
4


,자치구,동,센터명,동별강좌정원,사고건수,독거노인포함,대중교통,합계,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,위도UTMK,경도UTMK,디지털정보지수,주변특성
4,은평구,대조동,대조동 주민센터,230.0,0.0,6636327.0,0.0,1327,5001,4.811632,26.534693,1.956991e+06,949056.126411,26627.03,1.0


In [89]:
eunpyeong.columns

Index(['자치구', '동', '센터명', '동별강좌정원', '사고건수', '독거노인포함', '대중교통', '합계', '65세이상고령자',
       '전연령별독거노인비율', '노인인구별독거노인비율', '위도UTMK', '경도UTMK', '디지털정보지수', '주변특성'],
      dtype='object')

In [90]:
eunpyeong.iloc[4:5,:]

,자치구,동,센터명,동별강좌정원,사고건수,독거노인포함,대중교통,합계,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,위도UTMK,경도UTMK,디지털정보지수,주변특성
4,은평구,대조동,대조동 주민센터,230.0,0.0,6636327.0,0.0,1327,5001,4.811632,26.534693,1.956991e+06,949056.126411,26627.03,1.0


In [93]:
eunpyeong.iloc[3:4,:]

,자치구,동,센터명,동별강좌정원,사고건수,독거노인포함,대중교통,합계,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,위도UTMK,경도UTMK,디지털정보지수,주변특성
3,은평구,구산동,구산동 주민센터,240.0,0.0,7001176.0,0.0,1141,6136,3.45789,18.595176,1.956719e+06,948118.844539,31058.19,1.0


In [94]:
eunpyeong.iloc[2:3,:]

,자치구,동,센터명,동별강좌정원,사고건수,독거노인포함,대중교통,합계,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,위도UTMK,경도UTMK,디지털정보지수,주변특성
2,은평구,불광2동,불광2동 주민센터,70.0,0.0,10161333.0,0.0,1729,5877,6.016006,29.419772,1.958396e+06,949647.222171,25945.56,1.0


* 대중교통이 0이라서 기각.

In [95]:
eunpyeong.iloc[5:6,:]

,자치구,동,센터명,동별강좌정원,사고건수,독거노인포함,대중교통,합계,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,위도UTMK,경도UTMK,디지털정보지수,주변특성
5,은평구,응암3동,응암3동 주민센터,173.0,0.0,9144800.0,1.0,1633,5600,6.390639,29.160714,1.954613e+06,948610.636749,22766.28,2.0


In [ ]:
eunpyeong['동별강좌정원'].mean()

102.55555555555556

### 응암3동 주민센터가 동별강좌정원 수 많고 대중교통이 있어서 채택.

## 노원구 시작

In [43]:
dfnowon = dfclu2[dfclu2['자치구']=='노원구']
dfnowon.head(1)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
158,노원구,월계2동,월계2동 주민센터,0.0,0.0,0.0,0.0,550.0,0.0,9868194.0,...,5691,6.279196,30.469162,37.632348,127.050861,1.958998e+06,960559.81729,24685.51,74.0,0.0


In [44]:
# 인덱스 조정 작업.(158 -> 0)
dfnowon_data = dfnowon.reset_index()
nowon = dfnowon_data.iloc[:,1:]
nowon.head(3)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
0,노원구,월계2동,월계2동 주민센터,0.0,0.0,0.0,0.0,550.0,0.0,9868194.0,...,5691,6.279196,30.469162,37.632348,127.050861,1.958998e+06,960559.817290,24685.51,74.0,0.0
1,노원구,월계3동,월계3동 주민센터,0.0,0.0,0.0,0.0,656.0,0.0,8847188.0,...,5906,4.865216,25.364037,37.630706,127.063176,1.958811e+06,961645.641883,28589.37,72.0,0.0
2,노원구,공릉1동,공릉1동 주민센터,1.0,2.0,0.0,0.0,427.0,0.0,10619136.0,...,6336,4.373239,26.452020,37.624800,127.073833,1.958152e+06,962583.028005,36737.28,84.0,3.0


In [45]:
# 노원구중 각 주민센터(입지선정)과 다른 주민센터와의 거리합.
distance = []
distance2 = []
for i in range(len(nowon)):
    for j in range(len(nowon)):
        distance.append(np.sqrt((nowon['위도UTMK'][j] - nowon['위도UTMK'][i])**2 + (nowon['경도UTMK'][j] - nowon['경도UTMK'][i])**2))
    distance2.append(sum(distance))
    distance= []
distance2

[21170.384231536427,
 19481.02081533848,
 24032.438508341933,
 29924.66642260715,
 27614.435604267994,
 16739.90114964095,
 16381.14033944846,
 17348.27468083776]

In [46]:
# 노원구의 인구 * 거리 값.
nowonpopdist = nowon['독거노인포함']*distance2
print(nowonpopdist.sort_values())
print(nowonpopdist.argmin())
nowon.iloc[5:6,:]

5    9.489666e+10
7    9.834546e+10
1    1.723523e+11
0    2.089135e+11
2    2.552037e+11
6    2.778205e+11
3    2.819318e+11
4    3.033437e+11
Name: 독거노인포함, dtype: float64
5


,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
5,노원구,상계6.7동,상계6.7동주민센터,0.0,0.0,0.0,1.0,0.0,0.0,5668890.0,...,4866,3.462933,23.941636,37.654856,127.067016,1.961489e+06,961996.781873,32733.66,70.0,1.0


In [47]:
# 두번째로 거리 * 인구 낮은 행 확인
nowon.iloc[7:8,:11]

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,대중교통
7,노원구,상계6.7동,상계6.7동 노원구청,0.0,0.0,0.0,0.0,0.0,0.0,5668890.0,3.0


* 동별강좌정원이 0명이라 기각.

In [48]:
# 세번째로 거리 * 인구 낮은 행 확인
nowon.iloc[1:2,:11]

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,대중교통
1,노원구,월계3동,월계3동 주민센터,0.0,0.0,0.0,0.0,656.0,0.0,8847188.0,0.0


In [49]:
# 네번째로 거리 * 인구 낮은 행 확인
nowon.iloc[0:1,:11]

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,대중교통
0,노원구,월계2동,월계2동 주민센터,0.0,0.0,0.0,0.0,550.0,0.0,9868194.0,2.0


In [50]:
nowon.iloc[2:3,:11]
# 공릉1동주민센터로 해야하지 않았나?

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,대중교통
2,노원구,공릉1동,공릉1동 주민센터,1.0,2.0,0.0,0.0,427.0,0.0,10619136.0,1.0


### 월계2동 주민센터가 동별강좌정원 준수하고 대중교통이 2라서 채택!

## 관악구 시작

In [51]:
# 관악구 추출.
dfgwanak = dfclu2[dfclu2['자치구']=='관악구']
dfgwanak.head(1)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
321,관악구,은천동,은천동 주민센터,0.0,1.0,0.0,0.0,0.0,0.0,8733648.0,...,5838,4.324324,25.625214,37.485247,126.942418,1.942729e+06,950894.778879,33254.97,74.0,1.0


In [52]:
# 인덱스 조정 작업.(320 -> 0)
dfgwanak_data = dfgwanak.reset_index()
gwanak = dfgwanak_data.iloc[:,1:]
gwanak.head(3)

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,...,65세이상고령자,전연령별독거노인비율,노인인구별독거노인비율,X,Y,위도UTMK,경도UTMK,디지털정보지수,총합 9,주변특성
0,관악구,은천동,은천동 주민센터,0.0,1.0,0.0,0.0,0.0,0.0,8733648.0,...,5838,4.324324,25.625214,37.485247,126.942418,1.942729e+06,950894.778879,33254.97,74.0,1.0
1,관악구,청룡동,청룡동 주민센터,0.0,0.0,0.0,0.0,443.0,0.0,5942970.0,...,4455,3.672402,29.943883,37.479103,126.941637,1.942048e+06,950821.698729,39840.23,72.0,0.0
2,관악구,난곡동,난곡동 주민센터,0.0,0.0,0.0,0.0,436.0,0.0,8099684.0,...,5287,5.460897,28.976735,37.470930,126.921641,1.941152e+06,949048.179417,25578.71,72.0,0.0


In [53]:
# 관악구중 각 주민센터(입지선정)과 다른 주민센터와의 거리합.
distance = []
distance2 = []
for i in range(len(gwanak)):
    for j in range(len(gwanak)):
        distance.append(np.sqrt((gwanak['위도UTMK'][j] - gwanak['위도UTMK'][i])**2 + (gwanak['경도UTMK'][j] - gwanak['경도UTMK'][i])**2))
    distance2.append(sum(distance))
    distance= []
distance2

[9951.885579161859,
 8381.208939268095,
 9344.399584014203,
 11022.954277704599,
 12117.559862746628,
 10907.305204881177]

In [55]:
# 관악구의 인구 * 거리 값.
gwanakpopdist = gwanak['독거노인포함']*distance2
print(gwanakpopdist.sort_values())
print(gwanakpopdist.argmin())
gwanak.iloc[1:2,:11]

1    4.980927e+10
5    6.482179e+10
2    7.568668e+10
3    8.472968e+10
0    8.691627e+10
4    1.219623e+11
Name: 독거노인포함, dtype: float64
1


,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,대중교통
1,관악구,청룡동,청룡동 주민센터,0.0,0.0,0.0,0.0,443.0,0.0,5942970.0,0.0


* 대중교통이 0이라서 기각.

In [56]:
# 두번째로 거리*인구 적은 행 확인.
gwanak.iloc[5:6,:11]

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,대중교통
5,관악구,청룡동,청룡동 관악구청,0.0,0.0,0.0,0.0,443.0,3.0,5942970.0,2.0


In [60]:
gwanak.iloc[4:5,:11]

,자치구,동,센터명,공원수,도서관수,병원수,배움터수,동별강좌정원,사고건수,독거노인포함,대중교통
4,관악구,삼성동,삼성동 주민센터,0.0,0.0,0.0,0.0,418.0,0.0,10064925.0,2.0


### 관악구청이 동별강좌정원 수 많고 대중교통이 있어서 채택.

# 결론 
* 은평구 : 역촌동 주민센터 -> 응암3동 주민센터
* 노원구 : 상계1동 주민센터 -> 월계2동 주민센터.
* 관악구 : 삼성동 주민센터 -> 청룡동 관악구청.

* 기준이 강좌정원과 대중교통으로만 판단했으면 안됐음.
클러스터 한 기준컬럼 6개로 기준했어야 되지 않았나?하는 아쉬움이 듬.